In [48]:
%%writefile task4_mpi.cpp
// MPI код
#include <mpi.h>
#include <iostream>
#include <vector>
#include <numeric>

#define N 100000000  // 100 млн элементов

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int local_n = N / size;
    std::vector<int> local_data(local_n, 1);

    MPI_Barrier(MPI_COMM_WORLD);
    double start = MPI_Wtime();

    // локальная сумма
    long long local_sum = std::accumulate(
        local_data.begin(), local_data.end(), 0LL
    );

    long long global_sum = 0;

    // сбор суммы
    MPI_Reduce(
        &local_sum,
        &global_sum,
        1,
        MPI_LONG_LONG,
        MPI_SUM,
        0,
        MPI_COMM_WORLD
    );

    MPI_Barrier(MPI_COMM_WORLD);
    double end = MPI_Wtime();

    if (rank == 0) {
        std::cout << "Processes: " << size << std::endl;
        std::cout << "Global sum: " << global_sum << std::endl;
        std::cout << "MPI time (ms): " << (end - start) * 1000 << std::endl;
    }

    MPI_Finalize();
    return 0;
}


Writing task4_mpi.cpp


In [49]:
!mpic++ task4_mpi.cpp -O2 -o task4_mpi


In [53]:
!mpirun --allow-run-as-root --oversubscribe -np 2 ./task4_mpi

Processes: 2
Global sum: 100000000
MPI time (ms): 68.8975


In [54]:
!mpirun --allow-run-as-root --oversubscribe -np 4 ./task4_mpi


Processes: 4
Global sum: 100000000
MPI time (ms): 43.6581


In [55]:
!mpirun --allow-run-as-root --oversubscribe -np 8 ./task4_mpi


Processes: 8
Global sum: 100000000
MPI time (ms): 33.6668
